In [ ]:
import pandas as pd
import itertools
import random

from src.objects.customer import Customer
from src.aco.aco import ACO
from src.nearest_neighbor import NNO
import src.printer as pr

In [ ]:
def instance_generator(customer_count, rand_seed=0):
    """
    Generates a random instance of the vehicle routing problem.

    Returns:
    tuple: List of customer objects and the depot object., number of trucks, capacity of each truck
    """
    random.seed(rand_seed)  # Set the random seed for reproducibility
    
    customers = []
    
    # Number of trucks
    truck_count = random.randint(10, 75)
    
    # Capacity of each truck
    capacity = random.randint(10, 50) * 10

    # Number of customers
    num_customers = customer_count
    
    # Generate random service time for the customer
    service_time = random.randint(1, 5) * 10

    for i in range(1, num_customers + 1):

        # Generate random demand for the customer
        demand = random.randint(1, 10) * 10
        
        # Generate random coordinates for the customer
        x_coord = random.uniform(0, 100)
        y_coord = random.uniform(0, 100)

        # Generate random ready time for the customer
        ready_time = random.randint(0, 1000)

        # Generate random due date for the customer
        due_date = ready_time + random.randint(100, 300)

        # Create a Customer object and add it to the list of customers
        customers.append(Customer(i, x_coord, y_coord, demand, ready_time, due_date, service_time))

    # Generate the depot (warehouse) at the center of the customers
    depot = Customer(0, sum(customer.x_coord for customer in customers) / num_customers, sum(customer.y_coord for customer in customers) / num_customers, 0, 0, max(x.due_date for x in customers) + service_time + 100, 0)

    return [depot] + customers, truck_count, capacity

In [ ]:
def read_dataset(file_path):
    print(f"Reading dataset -> '{file_path}'")
    # Reading the vehicle data (lines 3-4)
    vehicle_df = pd.read_csv(file_path, skiprows=4, nrows=1, sep='\\s+', names=['Number', 'Capacity'])
    
    # Reading the customer data (after line 9)
    customers_df = pd.read_csv(file_path, skiprows=9, sep='\\s+', names=['Cust No.', 'XCoord.', 'YCoord.', 'Demand', 'Ready Time', 'Due Date', 'Service Time'])

    truck_count = vehicle_df['Number'][0]  # Get the number of trucks
    capacity = vehicle_df['Capacity'][0]  # Get the capacity of each truck

    # Create a list of Customer objects from the customer data
    customers = [Customer(row['Cust No.'], row['XCoord.'], row['YCoord.'], row['Demand'], row['Ready Time'], row['Due Date'], row['Service Time']) for index, row in customers_df.iterrows()]
    
    return customers, truck_count, capacity

In [ ]:
def main_aco_run(file_path, num_ants, num_iterations, pheromon_importance_alpha, heuristic_importance_beta, evaporation_rho, pheromon_init=-1):
    customers, truck_count, capacity = read_dataset(file_path)  # Read the dataset from the input file
    
    if pheromon_init == -1:
        nno = NNO(customers, truck_count, capacity)
        distance = nno.run()
        pheromon_init = 1/distance

    # Print the vehicle data
    print(f"Number of trucks: {truck_count}")
    print(f"Capacity of each truck: {capacity}")
    print(f"Number of customers: {len(customers) - 1}")

    # Initialize the Ant Colony Optimization (ACO) algorithm
    aco = ACO(
        customers.copy(),
        truck_count=truck_count,
        truck_capacity=capacity,
        iterations=num_iterations,
        ants_count=num_ants,
        pheromone_importance=pheromon_importance_alpha,
        heuristic_importance=heuristic_importance_beta,
        evaporation_rate=evaporation_rho,
        pheromone_init=pheromon_init,
        debug=False
    )

    # Run the ACO algorithm to find the best solution
    aco.run()
    
    return aco.get_results()

In [ ]:
def test_parameters_to_excel(file_path, excel_name, num_ants_list, num_iterations_list, pheromon_importance_alpha_list, heuristic_importance_beta_list, evaporation_rho_list, pheromon_init_list):
    customers, truck_count, capacity = read_dataset(file_path=file_path) # Read the dataset from the input file
    
    if -1 in pheromon_init_list:
        nno = NNO(customers, truck_count, capacity)
        distance = nno.run()
        pheromon_init_list[pheromon_init_list.index(-1)] = 1/distance
        
    # Print the vehicle data
    print(f"Number of trucks: {truck_count}")
    print(f"Capacity of each truck: {capacity}")
    print(f"Number of customers: {len(customers) - 1}")
    
    param_combinations = itertools.product(
        num_ants_list, num_iterations_list, pheromon_importance_alpha_list, heuristic_importance_beta_list, evaporation_rho_list, pheromon_init_list
    )
    
    (index_column,
     num_ants_column,
     num_iterations_column,
     pheromon_importance_alpha_column,
     heuristic_importance_beta_column,
     evaporation_rho_column,
     pheromon_init_column,
     truck_count_column,
     lowest_distance_column,
     average_distance_column,
     best_distance_column
     ) = [], [], [], [], [], [], [], [], [], [], []
    
    i = 1
    product_len = len(num_ants_list) * len(num_iterations_list) * len(pheromon_importance_alpha_list) * len(heuristic_importance_beta_list) * len(evaporation_rho_list) * len(pheromon_init_list)
    
    for num_ants, num_iterations, pheromon_importance_alpha, heuristic_importance_beta, evaporation_rho, pheromon_init in param_combinations:
        print(f"Iteration N°{i}/{product_len}")
        print("-" * 100)
        print(f"Ant count: {num_ants}")
        print(f"Iteration count: {num_iterations}")
        print(f"Pheromon importance (alpha): {pheromon_importance_alpha}")
        print(f"Heuristic importance (beta): {heuristic_importance_beta}")
        print(f"Evaporation rate (rho): {evaporation_rho}")
        print(f"Initial pheromone level: {pheromon_init}")        
        print("-" * 100)

        # Run the ACO algorithm with the current parameters
        (customers,
         depot, costs, best_solution, used_truck_count, best_distance, total_time) = main_aco_run(
            file_path=file_path,
            num_ants=num_ants,
            num_iterations=num_iterations,
            pheromon_importance_alpha=pheromon_importance_alpha,
            heuristic_importance_beta=heuristic_importance_beta,
            evaporation_rho=evaporation_rho,
            pheromon_init=pheromon_init)
        
        index_column.append(i)
        num_ants_column.append(num_ants)
        num_iterations_column.append(num_iterations)
        pheromon_importance_alpha_column.append(pheromon_importance_alpha)
        heuristic_importance_beta_column.append(heuristic_importance_beta)
        evaporation_rho_column.append(evaporation_rho)
        pheromon_init_column.append(pheromon_init)
        truck_count_column.append(len(best_solution))
        lowest_distance_column.append(min(costs))
        average_distance_column.append(sum(costs) / len(costs))
        best_distance_column.append(max(costs))

        # Print the truck usage details
        pr.print_truck_usage(best_solution, depot, show_graphics=False)
        
        i += 1
    
    excel_df = pd.DataFrame({"Ant count": num_ants_column,
                             "Iteration count": num_iterations_column,
                             "Pheromon importance (alpha)": pheromon_importance_alpha_column,
                             "Heuristic importance (beta)": heuristic_importance_beta_column,
                             "Evaporation rate (rho)": evaporation_rho_column,
                             "Initial pheromone level": pheromon_init_column,
                             "Minimum distance": lowest_distance_column,
                             "Average distance": average_distance_column,
                             "Maximum distance": best_distance_column})
    
    excel_file_path = f"results/{excel_name}-{file_path.split('/')[-1].split('.')[0]}.xlsx"
    excel_df.to_excel(excel_file_path, index=True)  # Save the results to an Excel file
    print(f"Results saved to {excel_file_path}")
  
def main():
    file = 'dataset/rc208.txt'
    (customers, depot, costs, best_solution, used_truck_count, total_distance, total_time) = main_aco_run(
        file_path=file,
        num_ants=100,
        num_iterations=100,
        pheromon_importance_alpha=1.0,
        heuristic_importance_beta=2.0,
        evaporation_rho=0.1,
        pheromon_init=-1)
    
    # Print the solution details
    pr.print_aco_solution(used_truck_count, total_distance, total_time)
    
    # Print optimal solution online
    pr.print_optimal_solution_online(file)

    # Plot the routes of the best solution
    pr.plot_routes(best_solution, depot)

    # Print the cost history
    pr.print_costs_history(costs)

    # Print the truck usage details
    pr.print_truck_usage(best_solution, depot, show_graphics=True)

def multiple_runs_analysis(data_file_path, excel_output_name, num_runs=50, show_graphics=False):
    
    excel_output_name = f"results/{excel_output_name}-{data_file_path.split('.')[0].split('/')[-1]}-{num_runs}.xlsx" # Output Excel file name with the name of the dataset
    
    distances_column = []

    customers, truck_count, capacity = read_dataset(data_file_path)  # Read the dataset from the input file

    # Print the vehicle data
    print(f"Number of trucks: {truck_count}")
    print(f"Capacity of each truck: {capacity}")
    print(f"Number of customers: {len(customers) - 1}")

    for i in range(num_runs):
        print(f"Run N°{i + 1}/{num_runs}")
        # Initialize the Ant Colony Optimization (ACO) algorithm
        (customers, depot, costs, best_solution, used_truck_count, best_distance, total_time) = main_aco_run(
        file_path=data_file_path,
        num_ants=30,
        num_iterations=40,
        pheromon_importance_alpha=1.0,
        heuristic_importance_beta=2.0,
        evaporation_rho=0.1,
        pheromon_init=-1)

        distances_column.append(best_distance)

        if show_graphics:
            # Plot the routes of the best solution
            pr.plot_routes(best_solution, depot)
        
        # Print the solution details
        pr.print_aco_solution(used_truck_count, best_distance, total_time)
    
    excel_df = pd.DataFrame({"File": [data_file_path] * len(distances_column), "Shortest Path": distances_column})
    excel_df.to_excel(excel_output_name, index=True)  # Save the results to an Excel file
    
    print(f"Results saved to {excel_output_name}")

if __name__ == '__main__':
    # test_parameters_to_excel(
    #     file_path='dataset/c101.txt',
    #     excel_name='paramteres_result.xlsx',
    #     num_ants_list=[20, 50, 100],
    #     num_iterations_list=[20, 50, 100],
    #     pheromon_importance_alpha_list=[1.0, 2.0, 3.0],
    #     heuristic_importance_beta_list=[1.0, 2.0, 3.0],
    #     evaporation_rho_list=[0.1, 0.3, 0.5],
    #     pheromon_init_list=[-1]
    # )
#     
    main()
#     
#     multiple_runs_analysis('dataset/rc208.txt', 'results_analysis', num_runs=100)